# Code to mount Google Drive

In [ ]:
# mount google drive to colab
from google.colab import drive
import os

n = 140
drive.mount('/content/drive')
print("#" * n)

# change working directory
print("### colabs default working directory")

print("###\n### path: {}".format(os.getcwd()))
print("#" * n)
print("### setted working directory")

%cd drive/Shareddrives/MADS_Milestone2/


print("###\n### {}".format(os.getcwd()))
print("#" * n)

Mounted at /content/drive
############################################################################################################################################
### colabs default working directory
###
### path: /content
############################################################################################################################################
### setted working directory
/content/drive/Shareddrives/MADS_Milestone2
###
### /content/drive/Shareddrives/MADS_Milestone2
############################################################################################################################################


# Import libraries

In [ ]:
from platform import python_version
import sys
import pandas as pd
import numpy as np
import pandas as pd
import networkx as nx
from matplotlib import pyplot as plt

print("used python version {}".format(python_version()))

used python version 3.7.12


## import all final input files to integrate the different network layers


In [ ]:
# current working directory
DIR = "/content/drive/Shareddrives/MADS_Milestone2/"

# get all the file names of data
files = os.listdir(DIR+"/res")

print("#"*140, "\n### ", files)

###########################################
# define abbreviations of diferent datasets
# c_  = curated
# gda = gene - disease association
# vda = variant - disease association
# dda= disease - disease association
###########################################

final_files = [s for s in files if "final" in s]
names = ["pp", "ip", "dp"]

DF = {n :pd.read_csv("{}/res/{}".format(DIR, f)).drop(columns='Unnamed: 0') for f,n in zip(final_files,names)}
DF

############################################################################################################################################ 
###  ['processed_protein_protein_interactions.csv', 'final_protein_protein_interaction_network.csv', 'final_disease_protein_interaction_network.csv', 'final_drug_protein_interaction_network.csv', 'INTEGRATED_FINAL_NETWORK.csv']


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


{'dp':                node1 node2  ... yearOfFirstApproval withdrawnNotice
 0        CHEMBL14762  1020  ...                 NaN             NaN
 1      CHEMBL1201716  3480  ...              2005.0             NaN
 2      CHEMBL3989861  6532  ...              2013.0             NaN
 3         CHEMBL1118  6532  ...              2008.0             NaN
 4      CHEMBL1743080  8795  ...                 NaN             NaN
 ...              ...   ...  ...                 ...             ...
 12579  CHEMBL1631540  6532  ...                 NaN             NaN
 12580  CHEMBL4297187  2261  ...                 NaN             NaN
 12581  CHEMBL4297187  2260  ...                 NaN             NaN
 12582  CHEMBL4297187  2264  ...                 NaN             NaN
 12583  CHEMBL4297187  2263  ...                 NaN             NaN
 
 [12584 rows x 21 columns],
 'ip':            node1     node2  ... node2_type diseaseClass
 0              1  C0036341  ...    disease          F03
 1              

In [ ]:
# remove column color and shape
pp = DF["pp"].loc[:,DF["pp"].columns.isin(["color", "shape"]) == False].copy()
pp["node1_type_orig"] = pp["node1_type"]
pp["node2_type_orig"] = pp["node2_type"]

ip = DF["ip"].rename(columns = {"diseaseClass": "IndicationClass__filter"}).copy()
ip.node1 = ip.node1.astype("str")
ip["node1_type_orig"] = "protein"
ip["node2_type_orig"] = ip["node2_type"]

dp = DF["dp"]
dp["node1_type_orig"] = dp["node1_type"]
dp["node2_type_orig"] = "protein"


pp

,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_ensembl.protein,node2_ensembl.protein,node1_type_orig,node2_type_orig
0,381,375,ARF5,ARF1,protein,protein,ENSP00000000233,ENSP00000440005,protein,protein
1,381,9727,ARF5,RAB11FIP3,protein,protein,ENSP00000000233,ENSP00000262305,protein,protein
2,381,2108,ARF5,ETFA,protein,protein,ENSP00000000233,ENSP00000452762,protein,protein
3,381,23647,ARF5,ARFIP2,protein,protein,ENSP00000000233,ENSP00000484121,protein,protein
4,4074,26088,M6PR,GGA1,protein,protein,ENSP00000000412,ENSP00000341344,protein,protein
...,...,...,...,...,...,...,...,...,...,...
132513,51386,6231,EIF3L,RPS26,protein,protein,ENSP00000485663,ENSP00000348849,protein,protein
132514,51386,6230,EIF3L,RPS25,protein,protein,ENSP00000485663,ENSP00000435096,protein,protein
132515,51386,6233,EIF3L,RPS27A,protein,protein,ENSP00000485663,ENSP00000272317,protein,protein
132516,51386,6205,EIF3L,RPS11,protein,protein,ENSP00000485663,ENSP00000270625,protein,protein


In [ ]:
ip.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(ip.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).value_counts("node_type")



node_type
disease_related_protein    9404
disease                    9347
dtype: int64

# exchange node_type of protein protein interaction network when its diseased

In [ ]:
# extract all indication related genes/proteins in protein protein interaction network
ind = pp.isin(list(set(ip.node1))).loc[:,["node1", "node2"]]

print("From the {} unique disease related proteins we can see below the number ob matched disease related genes in protein protein interaction network.".format(len(set(ip.node1))))
display(ind.apply(lambda x: x.value_counts()))
print("we find {} unique disease related proteins for node1 and {} for node 2".format(len(set(pp[ind["node1"]].node1)), len(set(pp[ind["node2"]].node2))))

# index it for node1 and node2 gene/proteins
ix = pp[ind["node1"]].index
iy = pp[ind["node2"]].index

# replace node_type from protein to disease related protein
pp.loc[ix, "node1_type"] = "disease_related_protein"
pp.loc[iy, "node2_type"] = "disease_related_protein"

# output number of replaced node_types
print("node1: There were {} proteins which had to be changed in node_type from 'protein' to 'protein_related_ protein'\nnode2: There were {} proteins which had to be changed in node_type from 'protein' to 'protein_related_ protein'".format(len(ix), len(iy)))


From the 9404 unique disease related proteins we can see below the number ob matched disease related genes in protein protein interaction network.


,node1,node2
True,76588,75885
False,55930,56633


we find 6240 unique disease related proteins for node1 and 6239 for node 2
node1: There were 76588 proteins which had to be changed in node_type from 'protein' to 'protein_related_ protein'
node2: There were 75885 proteins which had to be changed in node_type from 'protein' to 'protein_related_ protein'


In [ ]:
# display(pp[(pp.node2 == "10013")].value_counts("node2_type"))
# display(pp[(pp.node1 == "10013")].value_counts("node1_type"))
# pp[(pp.node1 == "10013") | (pp.node2 == "10013")]


this looks like something went wrong. i would have expected much more disease related nodes that would match the protein protein interaction network. why do we map only ~1/10 of disease related proteins??? 


solved! the issue was that protein nodes of ip were in wrnog type 'int' instead of str


## add node attributes such as color and shape 

In [ ]:
# add node attributes to protein protein interaction network
pp["node1_color"] = "blue"
pp["node1_shape"] = "o"
pp["node2_color"] = "blue"
pp["node2_shape"] = "o"

pp.loc[ix, "node1_color"] = "red"
pp.loc[iy, "node2_color"] = "red"

# add edge filters 
pp["IndicationClass__filter"] = ""


ip.loc[:,"node1_color"] = "red"
ip.loc[:,"node2_color"] = "red"
ip.loc[:,"node1_shape"] = "o"
ip.loc[:,"node2_shape"] = "^"

ip

,node1,node2,node1_name,node2_name,node1_type,node2_type,IndicationClass__filter,node1_type_orig,node2_type_orig,node1_color,node2_color,node1_shape,node2_shape
0,1,C0036341,A1BG,Schizophrenia,disease_related_protein,disease,F03,protein,disease,red,red,o,^
1,2,C0002395,A2M,Alzheimer's Disease,disease_related_protein,disease,C10;F03,protein,disease,red,red,o,^
2,2,C0007102,A2M,Malignant tumor of colon,disease_related_protein,disease,C06;C04,protein,disease,red,red,o,^
3,2,C0011265,A2M,Presenile dementia,disease_related_protein,disease,C10;F03,protein,disease,red,red,o,^
4,2,C0011570,A2M,Mental Depression,disease_related_protein,disease,F01,protein,disease,red,red,o,^
...,...,...,...,...,...,...,...,...,...,...,...,...,...
60473,109580095,C0002875,HBB-LCR,Cooley's anemia,disease_related_protein,disease,C16;C15,protein,disease,red,red,o,^
60474,109580095,C0005283,HBB-LCR,beta Thalassemia,disease_related_protein,disease,C16;C15,protein,disease,red,red,o,^
60475,109580095,C0019025,HBB-LCR,Hemoglobin F Disease,disease_related_protein,disease,C16;C15,protein,disease,red,red,o,^
60476,109580095,C0085578,HBB-LCR,Thalassemia Minor,disease_related_protein,disease,C16;C15,protein,disease,red,red,o,^


# exchange node_type of protein protein interaction network when its a drug target

In [ ]:
# extract all drug related genes/proteins in protein protein interaction network
ind = pp.isin(list(set(dp.node2))).loc[:,["node1", "node2"]]

print("From the {} unique drug target proteins we can see below the number ob matched drug target proteins in protein protein interaction network.".format(len(set(dp.node2))))
display(ind.apply(lambda x: x.value_counts()))
print("we find {} unique drug targets for node1 and {} for node 2".format(len(set(pp[ind["node1"]].node1)), len(set(pp[ind["node2"]].node2))))

# index it for node1 and node2 gene/proteins
ix = pp[ind["node1"]].index
iy = pp[ind["node2"]].index

# replace node_type from protein to disease related protein
pp.loc[ix, "node1_type"] = "drug_target"
pp.loc[iy, "node2_type"] = "drug_target"

# output number of replaced node_types
print("node1: There were {} proteins which had to be changed in node_type from 'protein' to 'drug_target'\nnode2: There were {} proteins which had to be changed in node_type from 'protein' to 'drug_target'".format(len(ix), len(iy)))


pp.loc[ix, "node1_color"] = "green"
pp.loc[iy, "node2_color"] = "green"




From the 1431 unique drug target proteins we can see below the number ob matched drug target proteins in protein protein interaction network.


,node1,node2
False,106004,106157
True,26514,26361


we find 1104 unique drug targets for node1 and 1104 for node 2
node1: There were 26514 proteins which had to be changed in node_type from 'protein' to 'drug_target'
node2: There were 26361 proteins which had to be changed in node_type from 'protein' to 'drug_target'


In [ ]:
# extract all drug related genes/proteins in protein protein interaction network
ind = ip.isin(list(set(dp.node2))).loc[:,["node1", "node2"]]

print("From the {} unique drug target proteins we can see below the number ob matched drug target proteins in indication protein interaction network.".format(len(set(dp.node2))))
display(ind.apply(lambda x: x.value_counts()))
print("we find {} unique drug targets for node1 and {} for node 2".format(len(set(ip[ind["node1"]].node1)), len(set(ip[ind["node2"]].node2))))

# index it for node1 and node2 gene/proteins
ix = ip[ind["node1"]].index
iy = ip[ind["node2"]].index

# replace node_type from protein to disease related protein
ip.loc[ix, "node1_type"] = "drug_target"
ip.loc[iy, "node2_type"] = "drug_target"

# output number of replaced node_types
print("node1: There were {} proteins which had to be changed in node_type from 'disease_related_protein' to 'drug_target'\nnode2: There were {} proteins which had to be changed in node_type from 'disease_related_protein' to 'drug_target'".format(len(ix), len(iy)))


ip.loc[ix, "node1_color"] = "green"
ip.loc[iy, "node2_color"] = "green"

From the 1431 unique drug target proteins we can see below the number ob matched drug target proteins in indication protein interaction network.


,node1,node2
False,43926,60478.0
True,16552,NaN


we find 1182 unique drug targets for node1 and 0 for node 2
node1: There were 16552 proteins which had to be changed in node_type from 'disease_related_protein' to 'drug_target'
node2: There were 0 proteins which had to be changed in node_type from 'disease_related_protein' to 'drug_target'


In [ ]:
# display(pp[(pp.node2 == "10013")].value_counts("node2_type"))
# display(pp[(pp.node1 == "10013")].value_counts("node1_type"))
# pp[(pp.node1 == "10013") | (pp.node2 == "10013")]



# display(ip[(ip.node2 == "10013")].value_counts("node2_type"))
# display(ip[(ip.node1 == "10013")].value_counts("node1_type"))
# ip[(ip.node1 == "10013") | (ip.node2 == "10013")]


it seems as would the drug target proteins have high degree indicated by a lot of redudant nodes

In [ ]:
dp.rename(columns={"node2_name.1": "node1_name", "IndicationClass_filter": "IndicationClass__filter"}, inplace=True)


container = []
for c in list(set(dp.columns)-set(ip.columns)):
  container += [c]
  ip[c] = ""

print("These columns were missing in the indication-protein interaction network")
print(container)

container = []
for c in list(set(dp.columns)-set(pp.columns)):
  container += [c]
  pp[c] = ""


print("These columns were missing in the protein-protein interaction network")
print(container)

print(ip.shape, pp.shape, dp.shape)



These columns were missing in the indication-protein interaction network
['blackBoxWarning', 'drugType', 'ActionType_drug__filter', 'tradeNames', 'hasBeenWithdrawn', 'isApproved', 'withdrawnNotice', 'yearOfFirstApproval', 'maximumClinicalTrialPhase', 'DrugTargetType_drug__filter']
These columns were missing in the protein-protein interaction network
['blackBoxWarning', 'drugType', 'ActionType_drug__filter', 'tradeNames', 'hasBeenWithdrawn', 'isApproved', 'withdrawnNotice', 'yearOfFirstApproval', 'maximumClinicalTrialPhase', 'DrugTargetType_drug__filter']
(60478, 23) (132518, 25) (12584, 23)


In [ ]:
set(pp.columns)-set(dp.columns)

{'node1_ensembl.protein', 'node2_ensembl.protein'}

In [ ]:
# merge the protein protein interactions and disease related proteins

df_org = pp.loc[:,pp.columns.isin(["node1_ensembl.protein",	"node2_ensembl.protein"])==False].append(ip, ignore_index=True).copy()
df_org = df_org.append(dp, ignore_index=True).copy()

# there are 1446 node2 items which are int -- need to convert them to str
df_org.node2 = [str(s) for s in df_org.node2.to_list()]

# display(df_org.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(df_org.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).value_counts("node_type"))
# display(df_org.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(df_org.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).count())

df_org.to_csv(DIR + "res/INTEGRATED_FINAL_NETWORK.csv", index=False)

print("Number of entities in original df")
display(df_org.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(df_org.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).value_counts("node_type"))

print("Number of original entities in original df")
display(df_org.rename(columns={"node1": "node", "node1_type_orig": "node_type_orig"}).loc[:,["node", "node_type_orig"]].append(df_org.rename(columns={"node2": "node", "node2_type_orig": "node_type_orig"}).loc[:,["node", "node_type_orig"]]).drop_duplicates(subset=["node"]).value_counts("node_type_orig"))

# display(df_org.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(df_org.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).count())

# display(len(set(df_org[df_org.node1_type == "protein"].node1.tolist() + df_org[df_org.node2_type == "protein"].node2.tolist())))

# orig_set = set(df_org[df_org.node1_type == "protein"].node1.tolist() + df_org[df_org.node2_type == "protein"].node2.tolist())

dtp = {c: str for c in df_org.columns}
DIR = "/content/drive/Shareddrives/MADS_Milestone2/"
df_file = pd.read_csv(DIR + "res/INTEGRATED_FINAL_NETWORK.csv", dtype=dtp)


print("Number of entities in loaded file df")
display(df_file.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(df_file.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).value_counts("node_type"))

print("Number of original entities in loaded file df")
display(df_file.rename(columns={"node1": "node", "node1_type_orig": "node_type_orig"}).loc[:,["node", "node_type_orig"]].append(df_file.rename(columns={"node2": "node", "node2_type_orig": "node_type_orig"}).loc[:,["node", "node_type_orig"]]).drop_duplicates(subset=["node"]).value_counts("node_type_orig"))
# df_file.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(df_file.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).count()



Number of entities in original df


node_type
disease                    9347
disease_related_protein    8222
protein                    4839
drug                       4369
drug_target                1431
dtype: int64

Number of original entities in original df


node_type_orig
protein    14492
disease     9347
drug        4369
dtype: int64

Number of entities in loaded file df


node_type
disease                    9347
disease_related_protein    8222
protein                    4839
drug                       4369
drug_target                1431
dtype: int64

Number of original entities in loaded file df


node_type_orig
protein    14492
disease     9347
drug        4369
dtype: int64

In [ ]:

dtp = {c: str for c in df_org.columns}
DIR = "/content/drive/Shareddrives/MADS_Milestone2/"
df_file = pd.read_csv(DIR + "res/INTEGRATED_FINAL_NETWORK.csv", dtype=dtp)
df_file.node2 = [str(s) for s in df_file.node2.to_list()] 


display(df_file.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(df_file.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).value_counts("node_type"))
df_file.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(df_file.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).count()


node_type
disease                    9347
disease_related_protein    8222
protein                    4839
drug                       4369
drug_target                1431
dtype: int64

node         28208
node_type    28208
dtype: int64

1446

In [ ]:
df_org[df_org.node2=="8353"].node2.tolist()

['8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353',
 '8353']

In [ ]:
df_file.shape, df_org.shape

((205580, 24), (205580, 23))

In [ ]:
file_set = set(df_file[df_file.node1_type == "protein"].node1.tolist() + df_file[df_file.node2_type == "protein"].node2.tolist())

read_miss = orig_set - file_set
print("there are {} missing proteins when read in".format(len(read_miss)))

df_org[df_org.isin(list(read_miss)).node2.tolist()]

there are 810 missing proteins when read in


,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_type_orig,node2_type_orig,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,DrugTargetType_drug__filter,blackBoxWarning,hasBeenWithdrawn,drugType,withdrawnNotice,isApproved,maximumClinicalTrialPhase,yearOfFirstApproval,tradeNames,ActionType_drug__filter
131072,8354,8353,HIST1H3I,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
131073,8354,8361,HIST1H3I,HIST1H4F,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
131074,8354,440686,HIST1H3I,HIST2H3PS2,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
131075,8354,8356,HIST1H3I,HIST1H3J,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
131076,8354,23408,HIST1H3I,SIRT5,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132513,51386,6231,EIF3L,RPS26,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
132514,51386,6230,EIF3L,RPS25,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
132515,51386,6233,EIF3L,RPS27A,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
132516,51386,6205,EIF3L,RPS11,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,


In [ ]:
df_file.iloc[df_org[df_org.node2=="8353"].index.tolist(),:]

,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_type_orig,node2_type_orig,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,DrugTargetType_drug__filter,blackBoxWarning,hasBeenWithdrawn,drugType,withdrawnNotice,isApproved,maximumClinicalTrialPhase,yearOfFirstApproval,tradeNames,ActionType_drug__filter
1869,23468,8353,CBX5,HIST1H3E,disease_related_protein,protein,protein,protein,red,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2866,23492,8353,CBX7,HIST1H3E,disease_related_protein,protein,protein,protein,red,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3260,11198,8353,SUPT16H,HIST1H3E,disease_related_protein,protein,protein,protein,red,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7659,25842,8353,ASF1A,HIST1H3E,disease_related_protein,protein,protein,protein,red,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8726,83444,8353,INO80B,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129614,8368,8353,HIST1H4L,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129919,8294,8353,HIST1H4I,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130698,83740,8353,H2AFB3,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
130753,3018,8353,HIST1H2BB,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
id = list(set(df_file[df_file.node2=="8353"].index.to_list()) - set(df_org[df_org.node2=="8353"].index.tolist()))
df_org.iloc[id,:]

,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_type_orig,node2_type_orig,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,DrugTargetType_drug__filter,blackBoxWarning,hasBeenWithdrawn,drugType,withdrawnNotice,isApproved,maximumClinicalTrialPhase,yearOfFirstApproval,tradeNames,ActionType_drug__filter
131072,8354,8353,HIST1H3I,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
131553,8968,8353,HIST1H3F,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
131458,8335,8353,HIST1H2AB,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
132196,8358,8353,HIST1H3B,HIST1H3E,disease_related_protein,protein,protein,protein,red,o,blue,o,,,,,,,,,,,
131785,8355,8353,HIST1H3G,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
132016,8352,8353,HIST1H3C,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
131287,1105,8353,CHD1,HIST1H3E,disease_related_protein,protein,protein,protein,red,o,blue,o,,,,,,,,,,,
132158,8367,8353,HIST1H4E,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,


In [ ]:
df_file[(df_org.node2 != df_file.node2)].node2

131072      8353
131073      8361
131074    440686
131075      8356
131076     23408
           ...  
132513      6231
132514      6230
132515      6233
132516      6205
132517    728689
Name: node2, Length: 1446, dtype: object

In [ ]:
df_org[(df_org.node2 != df_file.node2)].node2

131072      8353
131073      8361
131074    440686
131075      8356
131076     23408
           ...  
132513      6231
132514      6230
132515      6233
132516      6205
132517    728689
Name: node2, Length: 1446, dtype: object

In [ ]:
ip.node1.tolist()

['1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '9',
 '9',
 '9',
 '9',
 '9',
 '9',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '12',
 '12',
 '13',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '15',
 '16',
 '16',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '18',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '19',
 '21',
 '21',
 '21',
 '21',
 '21',
 '21',
 '21',
 '21',
 '21',
 '21',
 '21',
 '22',
 '22',
 '22',
 '23',
 '23',
 '23',
 '23',
 '24',
 '24',
 '24',
 '24',
 '24',
 '24',
 '24',
 '24',
 '24',
 '24',
 '24',
 '24',
 '24',
 '25',
 '25',
 '25',
 '25',
 '25',
 '25',
 '25',
 '25',
 '25',
 '25',
 '25',
 '25',
 '25',
 '25',
 '25',


In [ ]:
df_org.dtypes

node1                          object
node2                          object
node1_name                     object
node2_name                     object
node1_type                     object
node2_type                     object
node1_type_orig                object
node2_type_orig                object
node1_color                    object
node1_shape                    object
node2_color                    object
node2_shape                    object
IndicationClass__filter        object
DrugTargetType_drug__filter    object
blackBoxWarning                object
hasBeenWithdrawn               object
drugType                       object
withdrawnNotice                object
isApproved                     object
maximumClinicalTrialPhase      object
yearOfFirstApproval            object
tradeNames                     object
ActionType_drug__filter        object
dtype: object

In [ ]:
df_org[(df_org.node2 != df_file.node2)]

,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_type_orig,node2_type_orig,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,DrugTargetType_drug__filter,blackBoxWarning,hasBeenWithdrawn,drugType,withdrawnNotice,isApproved,maximumClinicalTrialPhase,yearOfFirstApproval,tradeNames,ActionType_drug__filter
131072,8354,8353,HIST1H3I,HIST1H3E,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
131073,8354,8361,HIST1H3I,HIST1H4F,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
131074,8354,440686,HIST1H3I,HIST2H3PS2,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
131075,8354,8356,HIST1H3I,HIST1H3J,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
131076,8354,23408,HIST1H3I,SIRT5,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132513,51386,6231,EIF3L,RPS26,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
132514,51386,6230,EIF3L,RPS25,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
132515,51386,6233,EIF3L,RPS27A,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,
132516,51386,6205,EIF3L,RPS11,protein,protein,protein,protein,blue,o,blue,o,,,,,,,,,,,


In [ ]:
display(df_org[df_org.apply(lambda x: x.str.contains(",")).node1_name.tolist()])
df_file.iloc[193317,:]

,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_type_orig,node2_type_orig,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,DrugTargetType_drug__filter,blackBoxWarning,hasBeenWithdrawn,drugType,withdrawnNotice,isApproved,maximumClinicalTrialPhase,yearOfFirstApproval,tradeNames,ActionType_drug__filter
193317,CHEMBL1201509,3973,"GONADOTROPIN, CHORIONIC",LHCGR,drug,drug_target,drug,drug_target,green,s,green,o,NaN,single protein,0,0,Protein,NaN,1,4,1973,"['A.p.l.', 'Choragon', 'Chorionic gonadotropin...",AGONIST
193326,CHEMBL1200874,3767,"TOLBUTAMIDE SODIUM, STERILE",KCNJ11,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein complex,0,0,Small molecule,NaN,1,4,1961,['Orinase diagnostic'],BLOCKER
193327,CHEMBL1200874,6833,"TOLBUTAMIDE SODIUM, STERILE",ABCC8,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein complex,0,0,Small molecule,NaN,1,4,1961,['Orinase diagnostic'],BLOCKER
194555,CHEMBL1201649,2100,"ESTROGENS, CONJUGATED",ESR2,drug,drug_target,drug,drug_target,green,s,green,o,C18;C05,protein family,1,0,Small molecule,NaN,1,4,1942,"['Cenestin', 'Oestrogen', 'Premarin', 'Premari...",AGONIST
194556,CHEMBL1201649,2099,"ESTROGENS, CONJUGATED",ESR1,drug,drug_target,drug,drug_target,green,s,green,o,C18;C05,protein family,1,0,Small molecule,NaN,1,4,1942,"['Cenestin', 'Oestrogen', 'Premarin', 'Premari...",AGONIST
195312,CHEMBL1201468,2100,"ESTROGENS, ESTERIFIED",ESR2,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,1,0,Small molecule,NaN,1,4,1977,"['Amnestrogen', 'Esterified estrogens', 'Estra...",AGONIST
195313,CHEMBL1201468,2099,"ESTROGENS, ESTERIFIED",ESR1,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,1,0,Small molecule,NaN,1,4,1977,"['Amnestrogen', 'Esterified estrogens', 'Estra...",AGONIST
195516,CHEMBL1200569,134,"CAFFEINE, CITRATED",ADORA1,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,0,0,Small molecule,NaN,1,4,1999,"['Cafcit', 'Caffeine citrate', 'Peyona']",ANTAGONIST
195517,CHEMBL1200569,136,"CAFFEINE, CITRATED",ADORA2B,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,0,0,Small molecule,NaN,1,4,1999,"['Cafcit', 'Caffeine citrate', 'Peyona']",ANTAGONIST
195518,CHEMBL1200569,140,"CAFFEINE, CITRATED",ADORA3,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,0,0,Small molecule,NaN,1,4,1999,"['Cafcit', 'Caffeine citrate', 'Peyona']",ANTAGONIST


node1                                                              CHEMBL1201509
node2                                                                       3973
node1_name                                               GONADOTROPIN, CHORIONIC
node2_name                                                                 LHCGR
node1_type                                                                  drug
node2_type                                                           drug_target
node1_type_orig                                                             drug
node2_type_orig                                                      drug_target
node1_color                                                                green
node1_shape                                                                    s
node2_color                                                                green
node2_shape                                                                    o
IndicationClass__filter     

In [ ]:
df_file[df_file.apply(lambda x: x.str.contains(",")).node1_name.tolist()]

,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_type_orig,node2_type_orig,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,DrugTargetType_drug__filter,blackBoxWarning,hasBeenWithdrawn,drugType,withdrawnNotice,isApproved,maximumClinicalTrialPhase,yearOfFirstApproval,tradeNames,ActionType_drug__filter
193317,CHEMBL1201509,3973,"GONADOTROPIN, CHORIONIC",LHCGR,drug,drug_target,drug,drug_target,green,s,green,o,NaN,single protein,0.0,0.0,Protein,NaN,1.0,4.0,1973.0,"['A.p.l.', 'Choragon', 'Chorionic gonadotropin...",AGONIST
193326,CHEMBL1200874,3767,"TOLBUTAMIDE SODIUM, STERILE",KCNJ11,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein complex,0.0,0.0,Small molecule,NaN,1.0,4.0,1961.0,['Orinase diagnostic'],BLOCKER
193327,CHEMBL1200874,6833,"TOLBUTAMIDE SODIUM, STERILE",ABCC8,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein complex,0.0,0.0,Small molecule,NaN,1.0,4.0,1961.0,['Orinase diagnostic'],BLOCKER
194555,CHEMBL1201649,2100,"ESTROGENS, CONJUGATED",ESR2,drug,drug_target,drug,drug_target,green,s,green,o,C18;C05,protein family,1.0,0.0,Small molecule,NaN,1.0,4.0,1942.0,"['Cenestin', 'Oestrogen', 'Premarin', 'Premari...",AGONIST
194556,CHEMBL1201649,2099,"ESTROGENS, CONJUGATED",ESR1,drug,drug_target,drug,drug_target,green,s,green,o,C18;C05,protein family,1.0,0.0,Small molecule,NaN,1.0,4.0,1942.0,"['Cenestin', 'Oestrogen', 'Premarin', 'Premari...",AGONIST
195312,CHEMBL1201468,2100,"ESTROGENS, ESTERIFIED",ESR2,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,1.0,0.0,Small molecule,NaN,1.0,4.0,1977.0,"['Amnestrogen', 'Esterified estrogens', 'Estra...",AGONIST
195313,CHEMBL1201468,2099,"ESTROGENS, ESTERIFIED",ESR1,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,1.0,0.0,Small molecule,NaN,1.0,4.0,1977.0,"['Amnestrogen', 'Esterified estrogens', 'Estra...",AGONIST
195516,CHEMBL1200569,134,"CAFFEINE, CITRATED",ADORA1,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,0.0,0.0,Small molecule,NaN,1.0,4.0,1999.0,"['Cafcit', 'Caffeine citrate', 'Peyona']",ANTAGONIST
195517,CHEMBL1200569,136,"CAFFEINE, CITRATED",ADORA2B,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,0.0,0.0,Small molecule,NaN,1.0,4.0,1999.0,"['Cafcit', 'Caffeine citrate', 'Peyona']",ANTAGONIST
195518,CHEMBL1200569,140,"CAFFEINE, CITRATED",ADORA3,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,0.0,0.0,Small molecule,NaN,1.0,4.0,1999.0,"['Cafcit', 'Caffeine citrate', 'Peyona']",ANTAGONIST


In [ ]:
df_org.apply(lambda x: x.str.contains(",")).sum()

In [ ]:
df_org[df_org.node2 == "8353"].shape

(109, 23)

In [ ]:
df_file[df_file.node2 == "8353"].shape

(117, 24)

In [ ]:
df_file[df_file.node2 == "8353"].shape

(117, 24)

In [ ]:
display(df.IndicationClass__filter[df.IndicationClass__filter.notnull()])
df_org.IndicationClass__filter[df_org.IndicationClass__filter.notnull()]

132518                F03
132519            C10;F03
132520            C06;C04
132521            C10;F03
132522                F01
               ...       
205536                C14
205537        C04;C20;C15
205560                F03
205561    C04;C20;C15;C14
205573                C10
Name: IndicationClass__filter, Length: 59072, dtype: object

0                        
1                        
2                        
3                        
4                        
               ...       
205536                C14
205537        C04;C20;C15
205560                F03
205561    C04;C20;C15;C14
205573                C10
Name: IndicationClass__filter, Length: 191590, dtype: object

In [ ]:
dtp = {c: str for c in df.columns}
DIR = "/content/drive/Shareddrives/MADS_Milestone2/"
df2 = pd.read_csv(DIR + "res/INTEGRATED_FINAL_NETWORK.csv", dtype=dtp)

df2.isnull().sum()

node1                               0
node2                               0
node1_name                          0
node2_name                          0
node1_type                          0
node2_type                          0
node1_type_orig                     0
node2_type_orig                     0
node1_color                         0
node1_shape                         0
node2_color                         0
node2_shape                         0
IndicationClass__filter        146508
DrugTargetType_drug__filter    192996
blackBoxWarning                192996
hasBeenWithdrawn               192996
drugType                       192996
withdrawnNotice                204990
isApproved                     192996
maximumClinicalTrialPhase      192996
yearOfFirstApproval            199004
tradeNames                     192996
ActionType_drug__filter        192996
dtype: int64

In [ ]:
10988+1985+7053
8222+5649+1431

15302

In [ ]:

DIR = "/content/drive/Shareddrives/MADS_Milestone2/"
df = pd.read_csv(DIR + "res/INTEGRATED_FINAL_NETWORK.csv")
df


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,12,13,16,17,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_type_orig,node2_type_orig,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,DrugTargetType_drug__filter,blackBoxWarning,hasBeenWithdrawn,drugType,withdrawnNotice,isApproved,maximumClinicalTrialPhase,yearOfFirstApproval,tradeNames,ActionType_drug__filter
0,381,375,ARF5,ARF1,protein,disease_related_protein,protein,protein,blue,o,red,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,381,9727,ARF5,RAB11FIP3,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,381,2108,ARF5,ETFA,protein,disease_related_protein,protein,protein,blue,o,red,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,381,23647,ARF5,ARFIP2,protein,protein,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4074,26088,M6PR,GGA1,disease_related_protein,disease_related_protein,protein,protein,red,o,red,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205575,CHEMBL1631540,6532,GSK163090,SLC6A4,drug,drug_target,drug,drug_target,green,s,green,o,NaN,single protein,0.0,0.0,Small molecule,NaN,0.0,2.0,NaN,[],ANTAGONIST
205576,CHEMBL4297187,2261,DERAZANTINIB,FGFR3,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,0.0,0.0,Small molecule,NaN,0.0,2.0,NaN,[],INHIBITOR
205577,CHEMBL4297187,2260,DERAZANTINIB,FGFR1,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,0.0,0.0,Small molecule,NaN,0.0,2.0,NaN,[],INHIBITOR
205578,CHEMBL4297187,2264,DERAZANTINIB,FGFR4,drug,drug_target,drug,drug_target,green,s,green,o,NaN,protein family,0.0,0.0,Small molecule,NaN,0.0,2.0,NaN,[],INHIBITOR


In [ ]:
# test = df.groupby(["node1"]).apply(lambda x: x.value_counts("node1_type"))
# 

In [ ]:
# test2 = test.reset_index()[test.reset_index().duplicated(subset="node1")]
# display(test2)
# test[test.reset_index()[["node1"]].isin(["10013"]).node1.tolist()]

,node1,node1_type,0


node1  node1_type 
10013  drug_target    62
dtype: int64

In [ ]:
pp.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(pp.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).value_counts("node_type")

node_type
protein                    5649
disease_related_protein    5297
drug_target                1104
dtype: int64

In [ ]:
df = pp.loc[:,pp.columns.isin(["node1_ensembl.protein",	"node2_ensembl.protein"])==False].append(ip, ignore_index=True)
df.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(df.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).value_counts("node_type")

node_type
protein                    10525
disease_related_protein     9404
disease                     9347
drug_target                 1104
dtype: int64

In [ ]:
df = df.append(dp, ignore_index=True)
df.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(df.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).value_counts("node_type")

node_type
protein                    10525
disease_related_protein     9404
disease                     9347
drug                        4369
drug_target                 1431
dtype: int64

In [ ]:
DIR = "/content/drive/Shareddrives/MADS_Milestone2/"
df = pd.read_csv(DIR + "res/INTEGRATED_FINAL_NETWORK2.csv").drop(columns="Unnamed: 0")
display(df)
df.rename(columns={"node1": "node", "node1_type": "node_type"}).loc[:,["node", "node_type"]].append(df.rename(columns={"node2": "node", "node2_type": "node_type"}).loc[:,["node", "node_type"]]).drop_duplicates(subset=["node"]).value_counts("node_type")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,11,12,15,16,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,DrugTargetType_drug__filter,blackBoxWarning,hasBeenWithdrawn,drugType,withdrawnNotice,isApproved,maximumClinicalTrialPhase,yearOfFirstApproval,tradeNames,ActionType_drug__filter
0,381,375,ARF5,ARF1,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,381,9727,ARF5,RAB11FIP3,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,381,2108,ARF5,ETFA,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,381,23647,ARF5,ARFIP2,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4074,26088,M6PR,GGA1,protein,protein,blue,o,blue,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205575,CHEMBL1631540,6532,GSK163090,SLC6A4,drug,drug_target,green,s,green,o,NaN,single protein,0.0,0.0,Small molecule,NaN,0.0,2.0,NaN,[],ANTAGONIST
205576,CHEMBL4297187,2261,DERAZANTINIB,FGFR3,drug,drug_target,green,s,green,o,NaN,protein family,0.0,0.0,Small molecule,NaN,0.0,2.0,NaN,[],INHIBITOR
205577,CHEMBL4297187,2260,DERAZANTINIB,FGFR1,drug,drug_target,green,s,green,o,NaN,protein family,0.0,0.0,Small molecule,NaN,0.0,2.0,NaN,[],INHIBITOR
205578,CHEMBL4297187,2264,DERAZANTINIB,FGFR4,drug,drug_target,green,s,green,o,NaN,protein family,0.0,0.0,Small molecule,NaN,0.0,2.0,NaN,[],INHIBITOR


node_type
protein                    12356
disease                     9347
disease_related_protein     6397
drug                        4369
drug_target                 1273
dtype: int64

node2_type
protein                    10135
disease                     9347
drug_target                 1431
disease_related_protein        1
dtype: int64

## brief networknx play around

In [ ]:
df

,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_color,node1_shape,node2_color,node2_shape,IndicationClass__filter,ActionType_drug__filter,drugType,yearOfFirstApproval,isApproved,DrugTargetType_drug__filter,withdrawnNotice,maximumClinicalTrialPhase,blackBoxWarning,hasBeenWithdrawn,tradeNames
0,381,375,ARF5,ARF1,protein,protein,blue,o,blue,o,,,,,,,,,,,
1,381,9727,ARF5,RAB11FIP3,protein,protein,blue,o,blue,o,,,,,,,,,,,
2,381,2108,ARF5,ETFA,protein,protein,blue,o,blue,o,,,,,,,,,,,
3,381,23647,ARF5,ARFIP2,protein,protein,blue,o,blue,o,,,,,,,,,,,
4,4074,26088,M6PR,GGA1,protein,protein,blue,o,blue,o,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205575,CHEMBL1631540,6532,GSK163090,SLC6A4,drug,drug_target,green,s,green,o,NaN,ANTAGONIST,Small molecule,NaN,0,single protein,NaN,2,0,0,[]
205576,CHEMBL4297187,2261,DERAZANTINIB,FGFR3,drug,drug_target,green,s,green,o,NaN,INHIBITOR,Small molecule,NaN,0,protein family,NaN,2,0,0,[]
205577,CHEMBL4297187,2260,DERAZANTINIB,FGFR1,drug,drug_target,green,s,green,o,NaN,INHIBITOR,Small molecule,NaN,0,protein family,NaN,2,0,0,[]
205578,CHEMBL4297187,2264,DERAZANTINIB,FGFR4,drug,drug_target,green,s,green,o,NaN,INHIBITOR,Small molecule,NaN,0,protein family,NaN,2,0,0,[]


In [ ]:
n1 = list(filter(lambda x: "node1" in x, df.columns)) 
n2 = list(filter(lambda x: "node2" in x, df.columns)) 


node1 = df.loc[:,n1].drop_duplicates(subset='node1', keep="last").copy()
node2 = df.loc[:,n2].drop_duplicates(subset='node2', keep="last").copy()
change = [''.join([i for i in c if not i.isdigit()]) for c in node1.columns]

node1.columns = change
node2.columns = change

node = node1.append(node2)

node = node.drop_duplicates(subset='node', keep="last")

In [ ]:

G = nx.from_pandas_edgelist(df, 'node1', 'node2', 
                            edge_attr=['node1_color', 'node2_color'])

# define node attributes
attrs = {r["node"]: {"color": r["node_color"], 
                               "shape": r["node_shape"], 
                               "name": r["node_name"], 
                               "type": r["node_type"]}
          
         for i,r in node.iterrows()}


# insert all node attributes
nx.set_node_attributes(G, attrs)


In [ ]:
colors=nx.get_node_attributes(G,'color')
shape=nx.get_node_attributes(G,'shape')

pos = nx.kamada_kawai_layout(G)

nx.draw_networkx_edges(G, pos, alpha=0.1)
nx.draw_networkx_nodes(G, pos, nodelist=[k for k in colors.keys()], 
                       node_color=[v for v in colors.values()], 
                       node_shape=[v for v in shape.values()],
                            node_size=100)

# nx.draw_networkx_labels(G, pos, {k:k for k in disease.keys()}, font_size=16, font_color='black')

# nx.draw_kamada_kawai(G, with_labels=False, font_weight='bold')
plt.show()

In [ ]:
dp = pd.read_pickle(DIR + "dat/networkdf_dp_test.data")

In [ ]:
dp

,node1,node2_ensembl.gene,node2,node2_name,color,shape,node1_type,node2_type,node1_name,indication_class
0,CHEMBL3989861,ENSG00000108576,6532,SLC6A4,yellow,?,drug,target,Desvenlafaxine fumarate,None
1,CHEMBL1118,ENSG00000108576,6532,SLC6A4,yellow,?,drug,target,Khedezla,None
2,CHEMBL2096642,ENSG00000172572,5139,PDE3A,yellow,?,drug,target,Inocor,None
3,CHEMBL12856,ENSG00000172572,5139,PDE3A,yellow,?,drug,target,WIN 40680,Cardiotonic
4,CHEMBL1200947,ENSG00000043591,153,ADRB1,yellow,?,drug,target,Metipranolol,None
...,...,...,...,...,...,...,...,...,...,...
1936,CHEMBL1201505,ENSG00000171557,2266,FGG,yellow,?,drug,target,Plasmin,None
1937,CHEMBL4297064,ENSG00000108576,6532,SLC6A4,yellow,?,drug,target,Toledomin,None
1938,CHEMBL259209,ENSG00000108576,6532,SLC6A4,yellow,?,drug,target,Milnacipran,None
1939,CHEMBL359744,ENSG00000131747,7153,TOP2A,yellow,?,drug,target,Rubex,Antineoplastic


In [ ]:
# !wget --directory-prefix=/content/drive/Shareddrives/MADS_Milestone2/dat/ https://www.disgenet.org/static/disgenet_ap1/files/downloads/disease_mappings.tsv.gz

In [ ]:
disease_mapping = pd.read_csv(DIR + "/dat/disease_mappings.tsv.gz", compression='gzip',  sep='\t') 

disease_mapping["convertion"] = disease_mapping.apply(lambda x:  x["vocabulary"] + "_" + x["code"]  , axis=1) 
disease_mapping = disease_mapping[disease_mapping.vocabulary == "EFO"]
disease_mapping[disease_mapping.convertion == "EFO_0000538"]

,diseaseId,name,vocabulary,code,vocabularyName,convertion
13995,C1832244,"CARDIOMYOPATHY, DILATED, 1C (disorder)",EFO,0000538,hypertrophic cardiomyopathy,EFO_0000538
13996,C3151204,"CARDIOMYOPATHY, FAMILIAL HYPERTROPHIC, 16",EFO,0000538,hypertrophic cardiomyopathy,EFO_0000538
13997,C3151267,"CARDIOMYOPATHY, FAMILIAL HYPERTROPHIC, 20",EFO,0000538,hypertrophic cardiomyopathy,EFO_0000538
13998,C0205700,Asymmetric Septal Hypertrophy,EFO,0000538,hypertrophic cardiomyopathy,EFO_0000538
13999,C2750467,"Cardiomyopathy, Familial Hypertrophic, 14",EFO,0000538,hypertrophic cardiomyopathy,EFO_0000538
14000,C2751427,"CARDIOMYOPATHY, FAMILIAL HYPERTROPHIC, 4, SUSC...",EFO,0000538,hypertrophic cardiomyopathy,EFO_0000538
14001,C0949658,"Cardiomyopathy, Hypertrophic, Familial",EFO,0000538,hypertrophic cardiomyopathy,EFO_0000538
14002,C4551472,Hypertrophic obstructive cardiomyopathy,EFO,0000538,hypertrophic cardiomyopathy,EFO_0000538
14003,C0007194,Hypertrophic Cardiomyopathy,EFO,0000538,hypertrophic cardiomyopathy,EFO_0000538
14004,C4748014,"CARDIOMYOPATHY, FAMILIAL HYPERTROPHIC 27",EFO,0000538,hypertrophic cardiomyopathy,EFO_0000538


In [ ]:
disease_mapping.value_counts("convertion")[disease_mapping.value_counts("convertion") > 1]

convertion
EFO_0000538    25
EFO_0001365    20
EFO_0004266    17
EFO_0001356    17
EFO_0003758    12
               ..
EFO_1001164     2
EFO_1000895     2
EFO_1001169     2
EFO_1001500     2
EFO_1001281     2
Length: 925, dtype: int64

In [ ]:
conv = disease_mapping.set_index("convertion").to_dict("diseaseId")


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


In [ ]:
# using pickle ensures correct dtypes 
mechanismOfAction = pd.read_pickle(DIR + 'dat/dfs/mechanismOfAction')
indication = pd.read_pickle(DIR + 'dat/dfs/indication')
molecule = pd.read_pickle(DIR + 'dat/dfs/molecule')
fda = pd.read_pickle(DIR + 'dat/dfs/fda')

In [ ]:
ind = [True if len(i) else False for i in indication.approvedIndications]
approved = indication[ind].explode("approvedIndications")

approved["UMLS"] = approved["approvedIndications"].replace(conv["diseaseId"])
approved

,id,indications,approvedIndications,indicationCount,UMLS
0,CHEMBL413,"[{'disease': 'EFO_0004599', 'efoName': 'acute ...",EFO_0003966,120,C0015397
0,CHEMBL413,"[{'disease': 'EFO_0004599', 'efoName': 'acute ...",EFO_0000540,120,C0021053
1,CHEMBL941,"[{'disease': 'MONDO_0044917', 'efoName': 't-ly...",EFO_0000616,117,C0027651
1,CHEMBL941,"[{'disease': 'MONDO_0044917', 'efoName': 't-ly...",EFO_0000339,117,C0023474
1,CHEMBL941,"[{'disease': 'MONDO_0044917', 'efoName': 't-ly...",EFO_0000311,117,C0347071
...,...,...,...,...,...
7971,CHEMBL3137312,"[{'disease': 'EFO_0004220', 'efoName': 'chroni...",EFO_0004220,6,C0524910
7971,CHEMBL3137312,"[{'disease': 'EFO_0004220', 'efoName': 'chroni...",EFO_0003047,6,C0019196
7971,CHEMBL3137312,"[{'disease': 'EFO_0004220', 'efoName': 'chroni...",EFO_0000763,6,C0042769
7975,CHEMBL1201260,"[{'disease': 'EFO_0000537', 'efoName': 'hypert...",EFO_0000537,1,C0020538


In [ ]:
dp = mechanismOfAction.explode("chemblIds").explode("targets").set_index(["chemblIds", "targets"]).copy()
print(dp.shape)
dp = dp[~dp.index.duplicated(keep='first')].reset_index().dropna(axis=0, subset=['chemblIds', 'targets'])

dp

(14663, 5)


,chemblIds,targets,actionType,mechanismOfAction,targetName,targetType,references
0,CHEMBL14762,ENSG00000164885,INHIBITOR,Cyclin-dependent kinase 5 inhibitor,Cyclin-dependent kinase 5,single protein,"[{'source': 'PubMed', 'ids': ['25747275', '262..."
1,CHEMBL1201716,ENSG00000140443,AGONIST,Insulin-like growth factor I receptor agonist,Insulin-like growth factor I receptor,single protein,"[{'source': 'DailyMed', 'ids': ['setid=a8b27a1..."
2,CHEMBL3989861,ENSG00000108576,INHIBITOR,Serotonin transporter inhibitor,Serotonin transporter,single protein,"[{'source': 'FDA', 'ids': ['label/2013/205208s..."
3,CHEMBL1118,ENSG00000108576,INHIBITOR,Serotonin transporter inhibitor,Serotonin transporter,single protein,"[{'source': 'FDA', 'ids': ['label/2013/205208s..."
6,CHEMBL1743080,ENSG00000120889,AGONIST,Tumor necrosis factor receptor superfamily mem...,Tumor necrosis factor receptor superfamily mem...,single protein,"[{'source': 'PubMed', 'ids': ['18304965', '201..."
...,...,...,...,...,...,...,...
13915,CHEMBL1631540,ENSG00000108576,ANTAGONIST,Serotonin transporter antagonist,Serotonin transporter,single protein,"[{'source': 'PubMed', 'ids': ['21053897', '210..."
13919,CHEMBL4297187,ENSG00000068078,INHIBITOR,Fibroblast growth factor receptor inhibitor,Fibroblast growth factor receptor,protein family,"[{'source': 'Other', 'ids': ['http://meetingli..."
13920,CHEMBL4297187,ENSG00000077782,INHIBITOR,Fibroblast growth factor receptor inhibitor,Fibroblast growth factor receptor,protein family,"[{'source': 'Other', 'ids': ['http://meetingli..."
13921,CHEMBL4297187,ENSG00000160867,INHIBITOR,Fibroblast growth factor receptor inhibitor,Fibroblast growth factor receptor,protein family,"[{'source': 'Other', 'ids': ['http://meetingli..."


In [ ]:
try:
    import mygene
except ModuleNotFoundError as err:
    !pip install mygene
    import mygene
    # Error handling
    print(err)
 
mg = mygene.MyGeneInfo()
out = mg.querymany(dp['targets'], scopes='ensembl.gene', fields='entrezgene,symbol', species='human', returnall=True)

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-12584...done.
Finished.
1090 input query terms found dup hits:
	[('ENSG00000164885', 7), ('ENSG00000140443', 20), ('ENSG00000108576', 66), ('ENSG00000120889', 7), (


there were no missing convertions

In [ ]:
ens_nonmatched = [i["query"] for i in out["out"] if "entrezgene" not in i.keys()]
"There are {} non matched enemblIDs due to the API and there are {} matches but there was no entrezID found".format(len(out["missing"]), len(ens_nonmatched))

'There are 0 non matched enemblIDs due to the API and there are 18 matches but there was no entrezID found'

In [ ]:
conv_entrez = {i["query"]: i["entrezgene"] for i in out["out"] if "entrezgene" in i.keys()}
conv_symbol = {i["query"]: i["symbol"] for i in out["out"] if "entrezgene" in i.keys()}
# conv_symbol


In [ ]:
dp["node2"] = dp.targets.replace(conv_entrez)
dp["node2_name"] = dp.targets.replace(conv_symbol)

dp["node1_type"] = "drug"
dp["node2_type"] = "drug_target"

dp["node1_color"] = "green"
dp["node2_color"] = "green"

dp["node1_shape"] = "s"
dp["node2_shape"] = "o"

dp["IndicationClass_indication__filter"] = "all"

dp.rename(columns={"chemblIds": "node1", "actionType": "ActionType_drug__filter", "targetType": "DrugTargetType_drug__filter", "mechanismOfAction": "node1_name" }, inplace=True)

dp = dp.loc[:, ["node1",	"node2",	"node1_name", 	"node2_name", 	"node1_type",	"node2_type",	"node1_color",	"node1_shape",	"node2_color",	"node2_shape", "ActionType_drug__filter", "IndicationClass_indication__filter", "DrugTargetType_drug__filter"]]

# dp.to_csv(DIR + "res/final_drug_protein_interaction_network.csv")

,node1,node2,node1_name,node2_name,node1_type,node2_type,node1_color,node1_shape,node2_color,node2_shape,ActionType_drug__filter,IndicationClass_indication__filter,DrugTargetType_drug__filter
0,CHEMBL14762,1020,Cyclin-dependent kinase 5 inhibitor,CDK5,drug,drug_target,green,s,green,o,INHIBITOR,all,single protein
1,CHEMBL1201716,3480,Insulin-like growth factor I receptor agonist,IGF1R,drug,drug_target,green,s,green,o,AGONIST,all,single protein
2,CHEMBL3989861,6532,Serotonin transporter inhibitor,SLC6A4,drug,drug_target,green,s,green,o,INHIBITOR,all,single protein
3,CHEMBL1118,6532,Serotonin transporter inhibitor,SLC6A4,drug,drug_target,green,s,green,o,INHIBITOR,all,single protein
6,CHEMBL1743080,8795,Tumor necrosis factor receptor superfamily mem...,TNFRSF10B,drug,drug_target,green,s,green,o,AGONIST,all,single protein
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13915,CHEMBL1631540,6532,Serotonin transporter antagonist,SLC6A4,drug,drug_target,green,s,green,o,ANTAGONIST,all,single protein
13919,CHEMBL4297187,2261,Fibroblast growth factor receptor inhibitor,FGFR3,drug,drug_target,green,s,green,o,INHIBITOR,all,protein family
13920,CHEMBL4297187,2260,Fibroblast growth factor receptor inhibitor,FGFR1,drug,drug_target,green,s,green,o,INHIBITOR,all,protein family
13921,CHEMBL4297187,2264,Fibroblast growth factor receptor inhibitor,FGFR4,drug,drug_target,green,s,green,o,INHIBITOR,all,protein family


In [ ]:
indication

,id,indications,approvedIndications,indicationCount
0,CHEMBL413,"[{'disease': 'EFO_0004599', 'efoName': 'acute ...","[EFO_0003966, EFO_0000540]",120
1,CHEMBL941,"[{'disease': 'MONDO_0044917', 'efoName': 't-ly...","[EFO_0000616, EFO_0000339, EFO_0000311]",117
2,CHEMBL1172928,"[{'disease': 'MONDO_0002009', 'efoName': 'majo...",[],1
3,CHEMBL1234886,"[{'disease': 'EFO_0003103', 'efoName': 'urinar...",[],69
4,CHEMBL3039596,"[{'disease': 'EFO_0000537', 'efoName': 'hypert...",[EFO_0000537],1
...,...,...,...,...
7986,CHEMBL2108209,"[{'disease': 'EFO_0000764', 'efoName': 'hiv in...",[],1
7987,CHEMBL2108029,"[{'disease': 'MONDO_0007254', 'efoName': 'brea...",[],2
7988,CHEMBL3989777,"[{'disease': 'MONDO_0003005', 'efoName': 'macu...",[],1
7989,CHEMBL1255794,"[{'disease': 'EFO_0000764', 'efoName': 'hiv in...",[],1


In [ ]:
if re.match("j.*", "jdjdjd"):
  print("hello")

hello


In [ ]:
import re
print(approved.shape)
# [i  for i in approved["IndicationClass__filter"] if re.match("\w\d{2,2}?;|\w\d{2,2}$", str(i))]


(5638, 6)


In [ ]:
# approved.UMLS.tolist()
import re
conv_indicationClass = ip.set_index("node2").to_dict()
approved["IndicationClass__filter"] = approved.UMLS.replace(conv_indicationClass["IndicationClass__filter"])
approved["IndicationClass__filter"] = [i if re.match("\w\d{2,2}?;|\w\d{2,2}$", str(i)) else "" for i in approved["IndicationClass__filter"] ]
conv_IndicationClass__filter = approved.set_index("id").to_dict()
# approved

dp["IndicationClass__filter"] = dp.node1.replace( approved.set_index("id").to_dict()["IndicationClass__filter"])
dp["IndicationClass__filter"] = [i if re.match("\w\d{2,2}?;|\w\d{2,2}$", str(i)) else "" for i in dp["IndicationClass__filter"] ]
dp


,id,indications,approvedIndications,indicationCount,UMLS,IndicationClass__filter
0,CHEMBL413,"[{'disease': 'EFO_0004599', 'efoName': 'acute ...",EFO_0003966,120,C0015397,
0,CHEMBL413,"[{'disease': 'EFO_0004599', 'efoName': 'acute ...",EFO_0000540,120,C0021053,
1,CHEMBL941,"[{'disease': 'MONDO_0044917', 'efoName': 't-ly...",EFO_0000616,117,C0027651,
1,CHEMBL941,"[{'disease': 'MONDO_0044917', 'efoName': 't-ly...",EFO_0000339,117,C0023474,
1,CHEMBL941,"[{'disease': 'MONDO_0044917', 'efoName': 't-ly...",EFO_0000311,117,C0347071,
...,...,...,...,...,...,...
7971,CHEMBL3137312,"[{'disease': 'EFO_0004220', 'efoName': 'chroni...",EFO_0004220,6,C0524910,C06;C01
7971,CHEMBL3137312,"[{'disease': 'EFO_0004220', 'efoName': 'chroni...",EFO_0003047,6,C0019196,C06;C01
7971,CHEMBL3137312,"[{'disease': 'EFO_0004220', 'efoName': 'chroni...",EFO_0000763,6,C0042769,
7975,CHEMBL1201260,"[{'disease': 'EFO_0000537', 'efoName': 'hypert...",EFO_0000537,1,C0020538,


In [ ]:
# print(dp.shape)
# print(approved.shape)
# dp["IndicationClass_indication__filter"] = dp.node1.replace( approved.set_index("id").to_dict()["IndicationClass_indication__filter"])
# dp["IndicationClass_indication__filter"] = [i if re.match("\w\d{2,2}?;|\w\d{2,2}$", str(i)) else "" for i in dp["IndicationClass_indication__filter"] ]
# dp
# dp.merge(approved.loc[:,["id", "IndicationClass_indication__filter"]], left_on='node1', right_on='id', how="left").shape

(12584, 14)
(5638, 6)


KeyError: ignored

In [ ]:
mol = molecule.drop(columns=["canonicalSmiles",	"inchiKey", "crossReferences", "synonyms",	"linkedTargets",	"linkedDiseases",	"description",	"childChemblIds", "parentId"]).copy()
dp.drop(columns="IndicationClass_indication__filter").merge(mol, left_on="node1", right_on="id", how="left").drop(columns= ["node1_name", "id"]).rename(columns={"name": "node2_name"}).to_csv(DIR + "res/final_drug_protein_interaction_network.csv")

In [ ]:
pd.concat([dp, approved.loc[:,["id", "IndicationClass__filter"]]], axis=1)

ValueError: ignored

In [ ]:
dp.shape

(12584, 13)

In [ ]:
conv_IndicationClass__filter["IndicationClass__filter"]

{'CHEMBL413': '',
 'CHEMBL941': '',
 'CHEMBL3039596': '',
 'CHEMBL1161': 'C08;C20;C09',
 'CHEMBL1751': '',
 'CHEMBL3989949': '',
 'CHEMBL4594214': '',
 'CHEMBL2109065': '',
 'CHEMBL2108581': '',
 'CHEMBL1272': 'C18;C19',
 'CHEMBL1200737': 'C16;C08',
 'CHEMBL597': '',
 'CHEMBL1224': '',
 'CHEMBL222813': '',
 'CHEMBL1743047': 'C04;C08',
 'CHEMBL4297536': '',
 'CHEMBL4297229': '',
 'CHEMBL1201229': 'C11',
 'CHEMBL427216': 'C18;C19',
 'CHEMBL508102': '',
 'CHEMBL2205807': '',
 'CHEMBL3187723': '',
 'CHEMBL1023': 'C04;C20;C15',
 'CHEMBL1201835': '',
 'CHEMBL493287': '',
 'CHEMBL1370': 'C08;C20;C09',
 'CHEMBL1534': '',
 'CHEMBL1734': '',
 'CHEMBL839': 'C11',
 'CHEMBL3707330': '',
 'CHEMBL2103737': 'C17',
 'CHEMBL2141296': '',
 'CHEMBL24171': '',
 'CHEMBL1604': '',
 'CHEMBL3989867': '',
 'CHEMBL1201127': '',
 'CHEMBL479': '',
 'CHEMBL3301598': '',
 'CHEMBL3039597': '',
 'CHEMBL34259': '',
 'CHEMBL1201825': '',
 'CHEMBL807': '',
 'CHEMBL89598': '',
 'CHEMBL461522': '',
 'CHEMBL3301589': '',
 '

In [ ]:
["" str(i) for i in approved.IndicationClass__filter]

['C0015397',
 'C0021053',
 'C0027651',
 'C0023474',
 'C0347071',
 'C0020538',
 'C0002103',
 'HP_0006536',
 'nan',
 'C0015230',
 'HP_0001742',
 'C08;C20;C09',
 'C23;C16;C14',
 'C14',
 'C14',
 'C0728936',
 'C10',
 'HP_0001250',
 'HP_0001945',
 'C0030193',
 'HP_0004419',
 'C0027651',
 'C0347071',
 'C1852091',
 'C18;C19',
 'C0035242',
 'C16;C08',
 'C0728936',
 'C1527304',
 'C01;C08',
 'C0042769',
 'C0027651',
 'C04;C08',
 'Orphanet_85447',
 'EFO_0005422',
 'C11',
 'C18;C19',
 'HP_0000820',
 'MONDO_0003472',
 'C0027651',
 'nan',
 'nan',
 'C04',
 'C0347071',
 'C0027651',
 'C04;C20;C15',
 'nan',
 'nan',
 'C0021053',
 'C0035435',
 'C0002103',
 'C0375359',
 'HP_0006536',
 'nan',
 'C08',
 'C01;C08',
 'C1969833',
 'HP_0001742',
 'C2675113',
 'C08;C20;C09',
 'C0015397',
 'C13;C12',
 'C16;C18;C15',
 'C18',
 'HP_0001891',
 'HP_0000020',
 'C23;C13;C12',
 'HP_0000103',
 'C11',
 'C0728936',
 'C11',
 'EFO_0006911',
 'C16;C18;C15',
 'C17',
 'C0027651',
 'C0347071',
 'nan',
 'C0004623',
 'C1836233',
 'C37

In [ ]:
[True if ".\d{3,}.*" in str(s) else False for s in approved["IndicationClass__filter"]]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [ ]:
# ind = [True if ("C17" in str(i).split(";")) | ("C20" in str(i).split(";"))  else False for i in DF["c_gda"].diseaseClass]#.value_counts() #.isin(["C17", "C20", "C17,C20", "C20,C17"])].value_counts("diseaseClass")

# DF["c_gda"][ind]
DF["c_gda"]["node1_type"] = "disease_related_protein"
df = DF["c_gda"].loc[:,["geneId", "diseaseId", "geneSymbol",  "diseaseName", "node1_type" ,"diseaseType", "diseaseClass"]
                ].rename(columns={"geneId":"node1", "diseaseId": "node2", "geneSymbol": "node1_name",  "diseaseName": "node2_name",
                                  "diseaseType": "node2_type"}).copy()
df[df.node2_type == "disease"].to_csv("res/final_disease_protein_interaction_network.csv")

In [ ]:
df = df[df.node2_type == "disease"]

# consider only diseases from class C17 (skin) or C20 (immune)
ind = [True if ("C17" in str(i).split(";")) | ("C20" in str(i).split(";"))  else False for i in df.diseaseClass]
df[ind].value_counts("node2_name").to_dict()

# First try to generate a network of disease genes (specific diseases)

In [ ]:
# specifiy disease of interest - serves as lookup to convert disease id with disease abrebiations
disease = {"AD": "C0011615", "Pso": "C0033860", "RA": "C0003873", "IBD": "C0010346", "Vitiligo": "C0042900", "Asthma": "C0004096", "AA": "C0002171", "HS": "C0162836"}

# generate an empty dataframe / Graph as container/placeholder
df = pd.DataFrame()
G = nx.Graph()

# loop through all diseases to get the nodes
for k,v in disease.items():
    if v not in set(DF["c_gda"].diseaseId):
        print({k:v})
    tmp = DF["c_gda"][DF["c_gda"].diseaseId == v]
    tmp["disease"] = k
    df = pd.concat([df, tmp])

# insert all edges into the empty graph with selected attributes
G = nx.from_pandas_edgelist(df, 'geneSymbol', 'disease', 
                            edge_attr=['score', 'EI', 'YearInitial', 'YearFinal', 'NofPmids', 'NofSnps', 'source'])

# define node attributes
g_attrs = {r[1]["geneSymbol"]: {"color": "red",
                                   "shape": "o",
                               "entrezID": r[1]["geneId"],
                               "DSI":  r[1]["DSI"],
                              "DPI":r[1]["DPI"],
                              } for r in df.iterrows()}

d_attrs = {r[1]["disease"]: {"color": "red",
                                   "shape": "^",
                               "diseaseId": r[1]["diseaseId"], 
                              'diseaseName': r[1]["diseaseName"],
                              "diseaseType": r[1]["diseaseType"],
                              "diseaseClass": r[1]["diseaseClass"],
                              "diseaseSemanticType": r[1]["diseaseSemanticType"],
                              "DSI":  r[1]["DSI"],
                              "DPI":r[1]["DPI"]
                              } for r in df.iterrows()}

attrs = dict(list(g_attrs.items()) + list(d_attrs.items()))

# insert all node attributes
nx.set_node_attributes(G, attrs)
                                                                         

# draw the first network

In [ ]:
nx.draw(G, with_labels=True)
plt.show()

In [ ]:
gda = DF["c_gda"].loc[:,["geneId", "diseaseId"]]

In [ ]:
sorted(nx.betweenness_centrality(G, normalized=True).items(),key=lambda x: x[1])
       

In [ ]:
sorted(dict(G.degree).items(), key=lambda x: x[1])

In [ ]:
G = nx.Graph()


for k,v in disease.items():
    tmp = DF["c_gda"][DF["c_gda"].diseaseId == v]
    
    genes = [(r[1]["geneSymbol"], {"color": "red",
                                   "shape": "o",
                               "entrezID": r[1]["geneId"],
                               "source": r[1]["source"],
                               "score": r[1]["score"]}) for r in tmp.iterrows()]
    
    genes_un = [(r[1]["geneSymbol"], {"color": "blue",
                                      "shape": "o",
                               "entrezID": r[1]["geneId"],
                               "source": r[1]["source"],
                               "score": r[1]["score"]}) for r in tmp.iterrows() if r[1]["geneSymbol"] not in list(G.nodes())]
    
#     genes_un = [(g, {"color": "blue"}) for g in tmp.geneSymbol.tolist() if g not in list(G.nodes())]
    dis   = (k, {"color": "red",
                "shape": "^",})
    G.add_nodes_from(genes_un + [dis])
#     print([(g, k) for g in genes[0]])
    G.add_edges_from([(g[0], k) for g in genes])


# try different plotting styles 

focus on just a subgraph (all AD associated genes)

In [ ]:
colors=nx.get_node_attributes(G,'color')

pos = nx.kamada_kawai_layout(G)

nx.draw_networkx_edges(G, pos, alpha=0.2)
nx.draw_networkx_nodes(G, pos, nodelist=[k for k in colors.keys() if k in list(disease.keys())], 
                       node_color=[v for k,v in colors.items() if k in list(disease.keys())], node_shape="^",
                            node_size=100)

nx.draw_networkx_nodes(G, pos, nodelist=[k for k in colors.keys() if k not in list(disease.keys())], 
                        node_color=[v for k,v in colors.items() if k not in list(disease.keys())], node_shape= "o",
                            node_size=100)
nx.draw_networkx_labels(G, pos, {k:k for k in disease.keys()}, font_size=16, font_color='black')

# nx.draw_kamada_kawai(G, with_labels=True, font_weight='bold')
plt.show()

consider only one disease with its disease associated genes and look what other diseases share the same disease associated genes

In [ ]:
dis_view = ["AD", "Pso", "IBD", "RA", "Asthma", "AA"]
G_sub = G.subgraph(list(set(list(sum([(e,E) for e,E in list(G.edges()) if "AD" in (e,E)], ())))) + dis_view)
colors=nx.get_node_attributes(G_sub,'color')
shape=nx.get_node_attributes(G_sub,'shape')

pos = nx.spring_layout(G_sub)

nx.draw_networkx_edges(G_sub, pos, alpha=0.2)

nx.draw_networkx_nodes(G_sub, pos, alpha=0.5, nodelist=[k for k in colors.keys() if k not in dis_view ], node_shape="o",
                       node_color=[v for k,v in colors.items() if k not in dis_view], node_size=500)

nx.draw_networkx_nodes(G_sub, pos, alpha=0.5, nodelist=[k for k in colors.keys() if k in dis_view ], node_shape="^",
                       node_color=[v for k,v in colors.items() if k in dis_view], node_size=800)

nx.draw_networkx_labels(G_sub, pos, {k:k for k in list(G_sub.nodes) }, font_size=10, font_color='black')

# nx.draw_kamada_kawai(G, with_labels=True, font_weight='bold')
plt.show()

In [ ]:
# AD asscoiated genes which are mapped directly to Pso, RA and IBD associtated genes
nx.draw(G.subgraph(list(set(list(sum([(e,E) for e,E in list(G.edges()) if "AD" in (e,E)], ())))) + dis_view), with_labels=True, font_weight='bold')
plt.show()

In [ ]:
!ls 

- should we remove disease–gene associations marked as therapeutic to prevent circularity in the analysis?
- should we filter disease–gene relationships to only consider genes whose protein products were present in the network of physical interactions between proteins?

# STRING DB


## Download physical protein protein interactions

In [ ]:
!wget --directory-prefix=/content/drive/Shareddrives/MADS_Milestone2/dat/ https://stringdb-static.org/download/protein.links.full.v11.5/9606.protein.links.full.v11.5.txt.gz
!wget --directory-prefix=/content/drive/Shareddrives/MADS_Milestone2/dat/ https://stringdb-static.org/download/protein.physical.links.full.v11.5/9606.protein.physical.links.full.v11.5.txt.gz
!wget --directory-prefix=/content/drive/Shareddrives/MADS_Milestone2/dat/ https://stringdb-static.org/download/protein.info.v11.5/9606.protein.info.v11.5.txt.gz


In [ ]:
info = pd.read_csv("dat/9606.protein.info.v11.5.txt.gz", compression='gzip', header=0, sep='\t', quotechar='"')

The keys (#string_protein_id) are enembl-protein ids instead of entrezID - we need to convert it to entrez id

In [ ]:
info

import the full network of protein protein interactions

In [ ]:
tmp = pd.read_csv("dat/9606.protein.links.full.v11.5.txt.gz", compression='gzip', header=0, sep=' ', quotechar='"')

In [ ]:
"There are {}Mio edges in the STRING DB".format(round(len(tmp) / 1000000, 1))

import the dataset that is mentioned to contain only physical interactions - to see if we are missing something

In [ ]:
tmp = pd.read_csv("dat/9606.protein.physical.links.full.v11.5.txt.gz", compression='gzip', header=0, sep=' ', quotechar='"')

In [ ]:
"There are {}Mio edges in the STRING DBconsidering only phsical interactions".format(round(len(tmp) / 1000000, 1))

In [ ]:
# clean node ids remove the homo sapiens code id
p1 = tmp.protein1.str.split(".", expand=True)
p2 = tmp.protein2.str.split(".", expand=True)

new = p1.merge(p2, how="inner", left_index=True, right_index=True)


In [ ]:
# check if the interactions with IL13 are real physical interactions
# this example i looked at IL13 = ENSP00000304915 cytokine which should bind to IL4RA IL13RA1 IL13RA2
IL13 = tmp[new.isin(["ENSP00000304915"]).sum(axis=1).replace({1: True, 0: False})]
IL13[IL13.experiments > 0]

I have learned something new. there is evidence that  ENSP00000457492 = TMEM219 dimerize with IL13RA2 and binds to IL13 but its not visible via STRINGDB gui what is going wrong here?

test another one - this time we look at at CXCL10 a chemokine  It seems it contains not only confident physical interactions -.-

In [ ]:
# check another one - if the interactions with CCL27 are real physical interactions
# this example i looked at CCL27 = ENSP00000219244 cytokine which should bind to CCR4 and CCR8
CCL27 = tmp[new.isin(["ENSP00000219244"]).sum(axis=1).replace({1: True, 0: False})]
CCL27[cxcl10.experiments > 0]

## BioGrid 

In [ ]:
!wget --directory-prefix=/content/drive/Shareddrives/MADS_Milestone2/dat/ https://downloads.thebiogrid.org/File/BioGRID/Release-Archive/BIOGRID-4.4.202/BIOGRID-ALL-4.4.202.tab3.zip

In [ ]:
tmp = pd.read_csv("dat/BIOGRID-ALL-4.4.202.tab3.zip", compression='zip', header=0, sep='\t', quotechar='"')

In [ ]:
pp = tmp[((tmp.loc[:,"Organism Name Interactor A"] == "Homo sapiens") & (tmp.loc[:,"Experimental System Type"] == "physical"))
        ].query('`Organism Name Interactor A` == `Organism Name Interactor B`')

In [ ]:
df = pd.DataFrame()
G2 = nx.Graph()

g_dis=nx.get_node_attributes(G,'entrezID')

indA = [v for k,v in g_dis.items() if v in pp.loc[:,"Entrez Gene Interactor A"].tolist()]
indB = [v for k,v in g_dis.items() if v in pp.loc[:,"Entrez Gene Interactor B"].tolist()]

part1 = pp.set_index("Entrez Gene Interactor A").loc[indA,:].reset_index()
part2 = pp.set_index("Entrez Gene Interactor B").loc[indB,:].reset_index()
protein = part1.append(part2)
protein["index"] = list(range(len(protein)+1,  len(protein)*2+1,1))
PRO = pd.concat([protein.loc[:,["Official Symbol Interactor A","Entrez Gene Interactor A"]].rename(columns={"Official Symbol Interactor A": "Official Symbol Interactor", 
                                                                                          "Entrez Gene Interactor A": "Entrez Gene Interactor"}), protein.loc[:,["Official Symbol Interactor B","Entrez Gene Interactor B", "index"]].set_index("index").rename(columns={"Official Symbol Interactor B": "Official Symbol Interactor", 
                                                                                          "Entrez Gene Interactor B": "Entrez Gene Interactor"})]
               ).drop_duplicates(subset=['Entrez Gene Interactor'])


    
G2 = nx.from_pandas_edgelist(PRO, 'geneSymbol', 'disease', 
                            edge_attr=['score', 'EI', 'YearInitial', 'YearFinal', 'NofPmids', 'NofSnps', 'source'])


                                                                         

In [ ]:
protein

In [ ]:
g_dis=nx.get_node_attributes(G,'entrezID')

indA = [v for k,v in g_dis.items() if v in pp.loc[:,"Entrez Gene Interactor A"].tolist()]
indB = [v for k,v in g_dis.items() if v in pp.loc[:,"Entrez Gene Interactor B"].tolist()]

part1 = pp.set_index("Entrez Gene Interactor A").loc[indA,:].reset_index()
part2 = pp.set_index("Entrez Gene Interactor B").loc[indB,:].reset_index()
protein = part1.append(part2)
protein["index"] = list(range(len(protein)+1,  len(protein)*2+1,1))
PRO = pd.concat([protein.loc[:,["Official Symbol Interactor A","Entrez Gene Interactor A", "Systematic Name Interactor A", "Synonyms Interactor A", "SWISS-PROT Accessions Interactor A",
                              "TREMBL Accessions Interactor A", "REFSEQ Accessions Interactor A"]

                            ].rename(columns={"Official Symbol Interactor A": "Official Symbol Interactor",
                                              "Entrez Gene Interactor A": "Entrez Gene Interactor",
                                             "Systematic Name Interactor A": "Systematic Name Interactor",
                                             "Synonyms Interactor A": "Synonyms Interactor",
                                             "SWISS-PROT Accessions Interactor A": "SWISS-PROT Accessions Interactor", 
                                             "TREMBL Accessions Interactor A": "TREMBL Accessions Interactor",
                                             "REFSEQ Accessions Interactor A": "REFSEQ Accessions Interactor"}),

                 protein.loc[:,["Official Symbol Interactor B","Entrez Gene Interactor B", "index", "Systematic Name Interactor B", "Synonyms Interactor B", 
                                "SWISS-PROT Accessions Interactor B", "TREMBL Accessions Interactor B", "REFSEQ Accessions Interactor B"]
                            ].set_index("index").rename(columns={"Official Symbol Interactor B": "Official Symbol Interactor",
                                                                 "Entrez Gene Interactor B": "Entrez Gene Interactor",
                                                                 "Systematic Name Interactor B": "Systematic Name Interactor",
                                                                 "Synonyms Interactor B": "Synonyms Interactor",
                                                                 "SWISS-PROT Accessions Interactor B": "SWISS-PROT Accessions Interactor", 
                                                                 "TREMBL Accessions Interactor B": "TREMBL Accessions Interactor",
                                                                 "REFSEQ Accessions Interactor B": "REFSEQ Accessions Interactor"})]
               ).drop_duplicates(subset=['Entrez Gene Interactor'])


remove = list(nx.get_node_attributes(G, "entrezID"))
PRO = PRO[~PRO['Entrez Gene Interactor'].isin(remove)]
PRO["color"] = "blue"
PRO["shape"] = "o"


G2 = G.copy()
G2 = nx.from_pandas_edgelist(protein, "Official Symbol Interactor A", 'Official Symbol Interactor B', 
                            edge_attr=['#BioGRID Interaction ID', 'Experimental System Type', 'Author', 'Publication Source', 'Throughput',
       'Score', 'Modification', 'Qualifications', 'Tags', 'Source Database', 'Ontology Term IDs', 'Ontology Term Names', 'Ontology Term Categories',
       'Ontology Term Qualifier IDs', 'Ontology Term Qualifier Names',
       'Ontology Term Types'])


nx.set_node_attributes(G2, PRO.set_index('Official Symbol Interactor').to_dict())

print(len(G.nodes()))
G.add_nodes_from(G2)
G.add_edges_from(G2.edges())
len(G.nodes())



In [ ]:
PRO = pd.concat([protein.loc[:,["Official Symbol Interactor A","Entrez Gene Interactor A", "Systematic Name Interactor A", "Synonyms Interactor A", "SWISS-PROT Accessions Interactor A",
                              "TREMBL Accessions Interactor A", "REFSEQ Accessions Interactor A"]

                            ].rename(columns={"Official Symbol Interactor A": "Official Symbol Interactor",
                                              "Entrez Gene Interactor A": "Entrez Gene Interactor",
                                             "Systematic Name Interactor A": "Systematic Name Interactor",
                                             "Synonyms Interactor A": "Synonyms Interactor",
                                             "SWISS-PROT Accessions Interactor A": "SWISS-PROT Accessions Interactor", 
                                             "TREMBL Accessions Interactor A": "TREMBL Accessions Interactor",
                                             "REFSEQ Accessions Interactor A": "REFSEQ Accessions Interactor"}),

                 protein.loc[:,["Official Symbol Interactor B","Entrez Gene Interactor B", "index", "Systematic Name Interactor B", "Synonyms Interactor B", 
                                "SWISS-PROT Accessions Interactor B", "TREMBL Accessions Interactor B", "REFSEQ Accessions Interactor B"]
                            ].set_index("index").rename(columns={"Official Symbol Interactor B": "Official Symbol Interactor",
                                                                 "Entrez Gene Interactor B": "Entrez Gene Interactor",
                                                                 "Systematic Name Interactor B": "Systematic Name Interactor",
                                                                 "Synonyms Interactor B": "Synonyms Interactor",
                                                                 "SWISS-PROT Accessions Interactor B": "SWISS-PROT Accessions Interactor", 
                                                                 "TREMBL Accessions Interactor B": "TREMBL Accessions Interactor",
                                                                 "REFSEQ Accessions Interactor B": "REFSEQ Accessions Interactor"})]
               ).drop_duplicates(subset=['Entrez Gene Interactor'])

# set(PRO.loc[:,"Official Symbol Interactor"].tolist()) 
PRO

In [ ]:
# remove = list(nx.get_node_attributes(G, "entrezID"))
# PRO[~PRO['Entrez Gene Interactor'].isin(remove)]

In [ ]:
# remove

In [ ]:
dis_view = ["AD"]
G_sub = G.subgraph(list(set(list(sum([(e,E) for e,E in list(G.edges()) if "AD" in (e,E)], ())))) + dis_view)
colors=nx.get_node_attributes(G_sub,'color')
shape=nx.get_node_attributes(G_sub,'shape')

pos = nx.spring_layout(G_sub)

nx.draw_networkx_edges(G_sub, pos, alpha=0.2)

nx.draw_networkx_nodes(G_sub, pos, alpha=0.5, nodelist=[k for k in colors.keys() if k not in dis_view ], node_shape="o",
                       node_color=[v for k,v in colors.items() if k not in dis_view], node_size=500)

nx.draw_networkx_nodes(G_sub, pos, alpha=0.5, nodelist=[k for k in colors.keys() if k in dis_view ], node_shape="^",
                       node_color=[v for k,v in colors.items() if k in dis_view], node_size=800)

nx.draw_networkx_labels(G_sub, pos, {k:k for k in list(G_sub.nodes) }, font_size=10, font_color='black')

# nx.draw_kamada_kawai(G, with_labels=True, font_weight='bold')
plt.show()

In [ ]:
dis_view = ["AD", "Pso", "IBD", "RA"]
G_sub = G.subgraph(list(set(list(sum([(e,E) for e,E in list(G.edges()) if "AD" in (e,E)], ())))) + dis_view)
colors=nx.get_node_attributes(G_sub,'color')
shape=nx.get_node_attributes(G_sub,'shape')

pos = nx.spring_layout(G_sub)

nx.draw_networkx_edges(G_sub, pos, alpha=0.2)

nx.draw_networkx_nodes(G_sub, pos, alpha=0.5, nodelist=[k for k in colors.keys() if k not in dis_view ], node_shape="o",
                       node_color=[v for k,v in colors.items() if k not in dis_view], node_size=500)

nx.draw_networkx_nodes(G_sub, pos, alpha=0.5, nodelist=[k for k in colors.keys() if k in dis_view ], node_shape="^",
                       node_color=[v for k,v in colors.items() if k in dis_view], node_size=800)

nx.draw_networkx_labels(G_sub, pos, {k:k for k in list(G_sub.nodes) }, font_size=10, font_color='black')

# nx.draw_kamada_kawai(G, with_labels=True, font_weight='bold')
plt.show()

In [ ]:
protein[protein.loc[:,"Official Symbol Interactor A" ]== "CXCL10"].loc[:,protein.columns[7:20]]


In [ ]:
protein.value_counts("Experimental System")

In [ ]:
import seaborn as sns
sns.displot(protein, x="Score", kind="kde")

In [ ]:
protein.Score = [str(i).replace("-", "0") for i in protein.Score]
protein["Score"] = protein.Score.astype(float)

In [ ]:
sns.displot(protein, x="Score", kind="kde")
plt.show()

In [ ]:
protein.value_counts("Score")

In [ ]:
g_dis=nx.get_node_attributes(G,'entrezID')


indA = [v for k,v in g_dis.items() if v in pp.loc[:,"Entrez Gene Interactor A"].tolist()]
indB = [v for k,v in g_dis.items() if v in pp.loc[:,"Entrez Gene Interactor B"].tolist()]

part1 = pp.set_index("Entrez Gene Interactor A").loc[indA,:].reset_index()
part2 = pp.set_index("Entrez Gene Interactor B").loc[indB,:].reset_index()
protein = part1.append(part2)
protein["index"] = list(range(len(protein)+1,  len(protein)*2+1,1))
PRO = pd.concat([protein.loc[:,["Official Symbol Interactor A","Entrez Gene Interactor A"]].rename(columns={"Official Symbol Interactor A": "Official Symbol Interactor", 
                                                                                          "Entrez Gene Interactor A": "Entrez Gene Interactor"}), protein.loc[:,["Official Symbol Interactor B","Entrez Gene Interactor B", "index"]].set_index("index").rename(columns={"Official Symbol Interactor B": "Official Symbol Interactor", 
                                                                                          "Entrez Gene Interactor B": "Entrez Gene Interactor"})]
               ).drop_duplicates(subset=['Entrez Gene Interactor'])


G.add_nodes_from([(r[1]["Official Symbol Interactor"],
                                      {"color": "blue",
                                       "shape": "o",
                                       "entrezID": r[1]["Entrez Gene Interactor"]}) for r in PRO.iterrows() if r[1]["Entrez Gene Interactor"] not in list(g_dis.values())])

print(len(G.edges()))

for r in protein.iterrows():
    if (r[1]["Entrez Gene Interactor A"], r[1]["Entrez Gene Interactor B"]) not in list(nx.edges(G)):
        G.add_edges_from([(r[1]["Official Symbol Interactor A"], r[1]["Official Symbol Interactor B"])])

print(len(G.edges()))


In [ ]:
len(G)

In [ ]:
protein.merge(PRO, how="inner", left_on="Entrez Gene Interactor B", right_on="Entrez Gene Interactor")

In [ ]:
len(G)

In [ ]:
pp.loc[:,pp.columns[10:37]].head()

In [ ]:
pp.value_counts("Experimental System")

In [ ]:
pp.columns

In [ ]:
pp.value_counts("Entrez Gene Interactor B")

In [ ]:
test = pd.read_excel("data/UniversityOfMichingan/Milestone_II/dat/41467_2021_21770_MOESM5_ESM.xlsx")

In [ ]:
test.shape

In [ ]:
test2 = pd.merge(pp, test,  how='inner', left_on=["Entrez Gene Interactor A","Entrez Gene Interactor B"], right_on = ["node_1","node_2"])

In [ ]:
test2.value_counts("SWISS-PROT Accessions Interactor B")

In [ ]:
pp.shape

In [ ]:
import os



In [ ]:
import os
dfs = [] # an empty list to store the data frames
file_list = os.listdir("NCT0080xxxx/")
for file in file_list:
    print("NCT0080xxxx/" + file)
    data = pd.read_json("NCT0080xxxx/" + file) # read data frame from json file
    dfs.append(data) # append the data frame to the list

temp = pd.concat(dfs, ignore_index=True) # concatenate all the data frames in the list.

NCT0080xxxx/NCT00801892.json
NCT0080xxxx/NCT00803127.json
NCT0080xxxx/NCT00807131.json
NCT0080xxxx/NCT00800033.json
NCT0080xxxx/NCT00806897.json
NCT0080xxxx/NCT00801099.json
NCT0080xxxx/NCT00802815.json
NCT0080xxxx/NCT00802802.json
NCT0080xxxx/NCT00808834.json
NCT0080xxxx/NCT00805844.json
NCT0080xxxx/NCT00805870.json
NCT0080xxxx/NCT00805259.json
NCT0080xxxx/NCT00801008.json
NCT0080xxxx/NCT00808535.json
NCT0080xxxx/NCT00804895.json
NCT0080xxxx/NCT00806741.json
NCT0080xxxx/NCT00800345.json
NCT0080xxxx/NCT00803257.json
NCT0080xxxx/NCT00807001.json
NCT0080xxxx/NCT00801710.json
NCT0080xxxx/NCT00808145.json
NCT0080xxxx/NCT00805025.json
NCT0080xxxx/NCT00800787.json
NCT0080xxxx/NCT00801502.json
NCT0080xxxx/NCT00809016.json
NCT0080xxxx/NCT00806533.json
NCT0080xxxx/NCT00800332.json
NCT0080xxxx/NCT00803907.json
NCT0080xxxx/NCT00803504.json
NCT0080xxxx/NCT00807625.json
NCT0080xxxx/NCT00806572.json
NCT0080xxxx/NCT00802269.json
NCT0080xxxx/NCT00800540.json
NCT0080xxxx/NCT00801164.json
NCT0080xxxx/NC

In [ ]:
DF

{'dp':                node1 node2  ... yearOfFirstApproval withdrawnNotice
 0        CHEMBL14762  1020  ...                 NaN             NaN
 1      CHEMBL1201716  3480  ...              2005.0             NaN
 2      CHEMBL3989861  6532  ...              2013.0             NaN
 3         CHEMBL1118  6532  ...              2008.0             NaN
 4      CHEMBL1743080  8795  ...                 NaN             NaN
 ...              ...   ...  ...                 ...             ...
 12579  CHEMBL1631540  6532  ...                 NaN             NaN
 12580  CHEMBL4297187  2261  ...                 NaN             NaN
 12581  CHEMBL4297187  2260  ...                 NaN             NaN
 12582  CHEMBL4297187  2264  ...                 NaN             NaN
 12583  CHEMBL4297187  2263  ...                 NaN             NaN
 
 [12584 rows x 21 columns],
 'ip':            node1     node2  ... node2_type diseaseClass
 0              1  C0036341  ...    disease          F03
 1              